# Glacier runoff using OGGM

In [ ]:
# geospatial
import geopandas as gpd
import xarray as xr

# oggm
from oggm import cfg, workflow, tasks, utils
from tqdm.notebook import tqdm

# basic
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from glob import glob

# import the MSsandbox modules
from MBsandbox.mbmod_daily_oneflowline import process_pmet_data, process_gcm_data
from MBsandbox.help_func import melt_f_calib_geod_prep_inversion
from MBsandbox.flowline_TIModel import run_from_climate_data_TIModel, run_with_hydro_daily

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
cfg.initialize(logging_level='ERROR', future = True)

cfg.PARAMS['use_multiprocessing']  = True
cfg.PARAMS['baseline_climate']     = ''
cfg.PARAMS['prcp_scaling_factor']  = 1
cfg.PARAMS['hydro_month_sh']       = 1
cfg.PARAMS['hydro_month_nh']       = 1
cfg.PARAMS['border']               = 80
cfg.PARAMS['geodetic_mb_period']   = '2000-01-01_2020-01-01'
cfg.PARAMS['store_model_geometry'] = True
cfg.PARAMS['continue_on_error']    = True
cfg.PARAMS['use_winter_prcp_factor'] = False
cfg.PARAMS['use_rgi_area'] = False
cfg.PARAMS['use_intersects'] = False
cfg.PARAMS['rgi_version'] = 7

param_dict = dict(t_melt  = 0, 
                  t_solid = 0,
                  t_liq   = 2)

variables = ["volume", "area", "melt_off_glacier_daily", "melt_on_glacier_daily", "liq_prcp_off_glacier_daily", "liq_prcp_on_glacier_daily"]
encoding  = {v: {'dtype': 'float32', 'complevel' : 5, 'zlib' : True} for v in variables}

In [ ]:
# Potential future scenario
gcm_list  = ["GFDL-ESM4", "IPSL-CM6A-LR", "MIROC6", "MPI-ESM1-2-LR", "MRI-ESM2-0"]
ssp_list  = ["ssp126", "ssp585"] 

## Calibration in the reference period

In [ ]:
# list with glacier geometries in RGI7 (and more data)
ids = gpd.read_file("/home/rooda/OneDrive/Patagonia/GIS South/Glaciers/RGI7_Hydro.shp")
ids_ref_mb21 = ids.set_index("RGIId").dmdtda_21

cfg.PATHS['working_dir']  = "/home/rooda/Hydro_results/oggm_runs_daily"
climate_file_path = "/home/rooda/Hydro_results/PMETsim_historical_OGGM.nc"

# init glacier directories
gdirs = workflow.init_glacier_directories(ids)

In [ ]:
# preprocessing tasks (same in "normal" OGGM)
workflow.execute_entity_task(tasks.define_glacier_region, gdirs, source="NASADEM");

task_list = [tasks.process_dem, 
             tasks.simple_glacier_masks, 
             tasks.elevation_band_flowline,
             tasks.fixed_dx_elevation_band_flowline, 
             tasks.compute_downstream_line,
             tasks.compute_downstream_bedshape]

for task in task_list:
    workflow.execute_entity_task(task, gdirs)

In [ ]:
# new function to process custom daily climate (at the monthly scale, it's the same)
workflow.execute_entity_task(process_pmet_data, gdirs,
                             temporal_resol='daily',
                             path = climate_file_path,
                             y0 = 1980, y1 = 2020, 
                             output_filesuffix="_PMET");

In [ ]:
# calibration using huggonet et al. 2021
workflow.execute_entity_task(melt_f_calib_geod_prep_inversion, gdirs,
                             pf=1,                    # precipitation factor
                             mb_type="mb_real_daily", # use daily temperature values
                             grad_type="cte",         # constant lapse rate (-6.5)
                             climate_type="PMET",     # climate type -> PMET
                             ye=2020,                 # last year
                             ref_mb=ids_ref_mb21,     # reference mb data
                             min_melt_f = 2,          # minimum melt factor that is allowed
                             max_melt_f = 1000,       # maximum melt factor that is allowed
                             kwargs_for_TIModel_Sfc_Type = param_dict);

In [ ]:
# save melt_f (not included in compile_glacier_statistics)
melt_f_ds = pd.DataFrame()   
for file_path in glob(cfg.PATHS['working_dir'] +  "/*/*/*/*/melt_f*.json", recursive = True):
    json_data = pd.read_json(file_path, lines = True)
    json_data["rgi_id"] = file_path.split('/')[8]
    melt_f_ds = pd.concat([melt_f_ds, json_data], ignore_index=True)

melt_f_ds.to_csv(cfg.PATHS['working_dir'] + "/glacier_statistics_melt_f.csv")

In [ ]:
# inversion by catchment (same in "normal OGGM")
for zone in range(1,10):
    ids_subset = ids[ids.ID_Zone == zone]
    gdirs_subset = [gdir for gdir in gdirs if gdir.rgi_id in ids_subset.RGIId.tolist()]

    workflow.calibrate_inversion_from_consensus(gdirs_subset, 
                                                volume_m3_reference = ids_subset.vol_M22.sum()*1e9, # Option to give an own total glacier volume to match to
                                                apply_fs_on_mismatch=True,     # on mismatch, try to apply an arbitrary value of fs
                                                error_on_mismatch=False,       # sometimes the given bounds do not allow to find a zero mismatch: this will normally raise an error
                                                filter_inversion_output=True); # apply terminus thickness filtering on the inversion output
# ready to use 
workflow.execute_entity_task(tasks.init_present_time_glacier, gdirs); 
utils.compile_glacier_statistics(gdirs);  # save statistics

In [ ]:
# run
workflow.execute_entity_task(run_with_hydro_daily, gdirs,
                             run_task = run_from_climate_data_TIModel, # any of the `run_*`` tasks in the MBSandbox.flowline_TIModel module.
                             ref_area_yr = 2000,                 # the hydrological output is computed over a reference area, specify here
                             ref_area_from_y0 = False,           # overwrite ref_area_yr to the first year of the timeseries
                             fixed_geometry_spinup_yr = 1990, 
                             ref_geometry_filesuffix = None,     # this kwarg allows to copy the reference area from a previous simulation
                             store_annual = True,               # whether to store annual outputs or only daily outputs
                             Testing = False,                    # if set to true, the 29th of February is set to nan values in non-leap years
                             ys=2000, ye=2020,                   # start and end years of the model run
                             climate_input_filesuffix="PMET",    # filesuffix for the input climate file                        
                             climate_filename='climate_historical',
                             output_filesuffix='_historical',    # for the output file
                             init_model_filesuffix = None,       # if you want to start from a previous model run state
                             mb_type="mb_real_daily",            # use daily temperature values
                             grad_type="cte",                    # constant lapse rate (-6.5)
                             precipitation_factor=1,             # multiply a factor to the precipitation time series
                             melt_f='from_json');                # calibrated melt_f

In [ ]:
utils.compile_run_output(gdirs, input_filesuffix='_historical');

## Climate projections for each glacier and scenario

In [ ]:
for gcm in tqdm(gcm_list):    
    for ssp in tqdm(ssp_list, leave = False):
        
        rid = gcm + "_" + ssp
        workflow.execute_entity_task(process_gcm_data, gdirs,
                                     temporal_resol='daily',
                                     path_prcp = "/home/rooda/Hydro_results/future_corrected/PP_{}_{}.nc".format(gcm,ssp),
                                     path_t2m  = "/home/rooda/Hydro_results/future_corrected/T2M_{}_{}.nc".format(gcm,ssp),
                                     path_hgt  = "/home/rooda/Hydro_results/PMETsim_historical_OGGM.nc",
                                     y0 = 2021, y1 = 2099,  output_filesuffix= "_daily_" + rid);

## Glacier projections for each glacier and scenario

In [ ]:
# future projections
for gcm in tqdm(gcm_list):    
    for ssp in tqdm(ssp_list, leave = False):

        rid = gcm + "_" + ssp

        for zone in tqdm(range(1,10), leave = False):
            
            ids_subset = ids[ids.ID_Zone == zone]
            gdirs_subset = [gdir for gdir in gdirs if gdir.rgi_id in ids_subset.RGIId.tolist()]
            
            workflow.execute_entity_task(run_with_hydro_daily, gdirs_subset,
                                         run_task = run_from_climate_data_TIModel, # any of the `run_*`` tasks in the MBSandbox.flowline_TIModel module.
                                         ref_area_from_y0 = True,                  # overwrite ref_area_yr to the first year of the timeseries
                                         ref_geometry_filesuffix = "_historical",  # this kwarg allows to copy the reference area from a previous simulation
                                         store_annual = True,                     # whether to store annual outputs or only daily outputs
                                         Testing = False,                          # if set to true, the 29th of February is set to nan values in non-leap years
                                         ys=2021, ye=2099,                         # start and end years of the model run
                                         climate_input_filesuffix= rid,            # filesuffix for the input climate file                        
                                         output_filesuffix= "_"+  rid,             # for the output file
                                         init_model_filesuffix = "_historical",    # if you want to start from a previous model run state
                                         climate_filename = 'gcm_data',
                                         mb_type="mb_real_daily",                  # use daily temperature values
                                         grad_type="cte",                          # constant lapse rate (-6.5)
                                         precipitation_factor=1,                   # multiply a factor to the precipitation time series
                                         melt_f='from_json');                      # calibrated melt_f (changed climate_type to "PMET")

            with utils.compile_run_output(gdirs_subset, path=False, input_filesuffix = "_" + rid) as ds:
                ds = ds[variables].astype("float32")
                ds.to_netcdf(cfg.PATHS['working_dir'] + "/run_output_" +  rid + "_" + str(zone) + ".nc", encoding=encoding)

            # I dont have enough space :(
            files = glob(cfg.PATHS['working_dir'] + "/per_glacier/*/*/*/model*" + rid + ".nc")
            
            for file in files: 
                os.remove(file)